In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
#import the european restaurant data
file_name = "../trip_advisor/tripadvisor_european_restaurants.csv"
import_csv = pd.read_csv(file_name)

C:\Users\julia\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#import Data into DF
euro_resto = pd.DataFrame(import_csv)

In [4]:
#drop columns not used
euro_resto_clean = euro_resto.drop(['restaurant_link','original_location', 'claimed', 'keywords', 
                              'atmosphere', 'price_range', 'popularity_detailed', 
                              'popularity_generic', 'top_tags', 'meals','special_diets', 'features', 
                              'vegetarian_friendly', 
                              'vegan_options','gluten_free', 'original_open_hours', 'open_days_per_week',
                              'open_hours_per_week', 'working_shifts_per_week',
                              'reviews_count_in_default_language', 'excellent', 'very_good',
                             'average', 'poor', 'terrible', 'food', 'service', 'value', 'atmosphere',
                             'keywords', 'default_language'], axis=1)


In [5]:
#filter the dataset to only include The british Island countries
britain_df = euro_resto_clean.loc[(euro_resto_clean['country'] == "England")| 
                                    (euro_resto_clean['country'] == "Ireland")|
                                   (euro_resto_clean['country'] == "Scotland")|
                                    (euro_resto_clean['country'] == "Wales")]


In [6]:
#remove any rows that have blank latitude, longitude or price_level
britain_df = britain_df.dropna(subset=['latitude', 'longitude', 'price_level'])

In [7]:
#truncate latitude to n decimal places
lat_list = britain_df['latitude'].tolist()

lat_list_trunc = []

for x in lat_list:
    n=2
    multiplier = 10**n
    value = int(x*(10**n))
    lat_list_trunc.append(value)

britain_df['lat_trunc'] = lat_list_trunc

In [8]:
#truncate longitude to n decimal places
long_list = britain_df['longitude'].tolist()

long_list_trunc = []

for x in long_list:
    n=2
    multiplier = 10**n
    value = int(x*(10**n))
    long_list_trunc.append(value)

britain_df['long_trunc'] = long_list_trunc

In [9]:
# convert both columns to string
britain_df['lat_trunc'] = britain_df['lat_trunc'].astype(str)
britain_df['long_trunc'] = britain_df['long_trunc'].astype(str)

In [10]:
#create location key from latitude and longitude truncated values
britain_df['location_key'] = britain_df[['lat_trunc', 'long_trunc']].agg('_'.join, axis=1)

In [11]:
#check number of duplicated values
total_list = len(britain_df['location_key'].tolist())
unique_list = len(britain_df['location_key'].unique().tolist())
dupes = britain_df['location_key'].duplicated().count()
print(total_list)
print(unique_list)
print((total_list-unique_list)/total_list*100)
print(dupes)


137465
33159
75.87822354781217
137465


In [12]:
britain_df = britain_df.drop(['lat_trunc','long_trunc'], axis=1)

### Correct the price_level

In [13]:
#check null values for price_level
britain_df['price_level'].notnull().count()

137465

In [14]:
#check how many characters need to be corrected
unique_price_level = britain_df['price_level'].unique()
unique_price_level

array(['€€-€€€', '€', '€€€€'], dtype=object)

In [15]:
#apply the map function to turn euro symbol to number
britain_df['price_level'] = britain_df['price_level'].map({'€':1,'€€-€€€':2, '€€€€':3 })


In [16]:
britain_df.head(5)

,restaurant_name,country,region,province,city,address,latitude,longitude,awards,price_level,cuisines,avg_rating,total_reviews_count,location_key
478379,Cafe Mylor,England,Cornwall,Falmouth,Mylor Churchtown,"Penarrow Road Mylor Yacht Harbour, Mylor Churc...",50.177025,-5.053397,"Travellers' Choice, Certificate of Excellence ...",2,"Cafe, British",4.5,347.0,5017_-505
478380,Subway,England,Surrey,NaN,Byfleet,"39 Old Woking Road, Byfleet KT14 6LG England",51.337010,-0.504781,NaN,1,Deli,3.5,4.0,5133_-50
478382,Basmati tandoori,England,Surrey,NaN,Byfleet,"23 High Road, Byfleet KT14 7QH England",51.339140,-0.478110,NaN,2,"Indian, Asian, Bangladeshi",3.0,4.0,5133_-47
478383,China Chef,England,Surrey,NaN,Byfleet,"72 High Road Byfleet, Byfleet KT14 7QL England",51.338920,-0.476430,NaN,2,Chinese,2.5,10.0,5133_-47
478384,The Plough,England,Surrey,NaN,Byfleet,"104 High Road, Byfleet KT14 7QT England",51.338780,-0.474000,NaN,2,"Pub, Bar, British",3.5,32.0,5133_-47


In [17]:
number_rows = britain_df.count()
number_rows

restaurant_name        137465
country                137465
region                 136948
province                43503
city                   115595
address                137465
latitude               137465
longitude              137465
awards                  55023
price_level            137465
cuisines               129548
avg_rating             133785
total_reviews_count    135156
location_key           137465
dtype: int64

In [18]:
# britain_df.loc[(britain_df['restaurant_name'] =='The Man Behind The Curtain' )]b
# # britain_df.loc[(britain_df['location_key'] =='491812_-21024' )]


In [19]:
#remove string words
words_to_remove = ['Restaurant', 'restaurant', 'The', 'the', 'bar', 'Bar', 'Inn']

for word in words_to_remove:
    britain_df['restaurant_name_clean'] = britain_df['restaurant_name'].str.replace(word,"")

In [20]:
britain_df.to_csv("Resources/british_islands_clean.csv")